## Random search compression on LeNet-5

In [1]:
PATH_PREFIX = '../../'
import sys
sys.path.append(PATH_PREFIX)

In [2]:
import torch
import torch.nn as nn
import pandas as pd
import os
import math

from data.mnist import MnistDataset
from data.utils.mnist_utils import *
from models.lenet.lenet import LeNet5
from utils.rnd import RandomController, Individual
from utils.weight_sharing import *
from utils.plot import *
from utils.fitness_controller import FitnessController

Setting parameters 

In [3]:
# net train settings
LEARNING_RATE = 0.0001
BATCH_SIZE = 32
N_CLASSES = 10
DEVICE = 'cpu'
EPOCHS = 100

# net save settings
NET_TYPE = 'relu'
NET_PATH = os.path.join(PATH_PREFIX, f'models/lenet/saves/lenet_{NET_TYPE}.save')

# dataset settings
DATA_PATH = os.path.join(PATH_PREFIX, 'data')

# random search iter count
NUM_INDIVIDUALS = 400

# random search search settings
REPR_RANGES = [range(1, 11) for _ in range(5)]

# random search save settings
SAVE_RND_FILE = os.path.join(PATH_PREFIX, 'results/test_RND_save.csv')
SAVE_EVERY = 1

# target position
TARGET = [1.0, 12.0]
TARGET_LOW_LIMIT = [0.95, 1.0]
LOCK_TARGET = False
TARGET_UPDATE_OFFSET = [0.001, 0.1]

# WS settings
SHARE_ORDER = [0, 1, 2, 3, 4]
RETRAIN_AMOUNT = None #[0, 0, 0, 0, 0]
PREC_REDUCT = 'f4'
CLUST_MOD_FOCUS = None #[0, 0, 0, 0, 0]
CLUST_MOD_SPREAD = None #[0, 0, 0, 0, 0]
CLUST_ALG = 'kmeans'

# range optimization settings
RANGE_OPTIMIZATION = True
RANGE_OPTIMIZATION_TRESHOLD = 0.97
RANGE_OPTIMIZATION_FILE = os.path.join(PATH_PREFIX, f'models/lenet/saves/lenet_{NET_TYPE}_layer_perf_{PREC_REDUCT}.csv')

Geting somewhat trained LeNet-5

In [4]:
dataset = MnistDataset(BATCH_SIZE, DATA_PATH, val_split=0.5)
model = LeNet5(N_CLASSES, NET_TYPE)
model.to(DEVICE)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
train_settings = [criterion, optimizer, dataset, EPOCHS, DEVICE, 1, True]

get_trained(model, NET_PATH, train_settings, DEVICE)

before_loss = get_accuracy(model, dataset.test_dl, DEVICE)

In [5]:
before_loss

0.9846

Setting weight share controller

In [ ]:
lam_opt = lambda mod : torch.optim.Adam(mod.parameters(), lr=LEARNING_RATE)
lam_train = lambda opt, epochs : train_net(model, criterion, opt, dataset, epochs, device=DEVICE)
lam_test = lambda : get_accuracy(model, dataset.test_dl, DEVICE)

ws_controller = WeightShare(model, lam_test, lam_opt, lam_train)
ws_controller.set_reset()

ws_controller.print_layers_info()

Defining fitness function

In [ ]:
def fitness_vals_fc(individual:Individual):
    # reset the net
    ws_controller.reset()
    
    # share weigts by particle
    if individual.data is None:
        individual.data = ws_controller.share(individual.representation, SHARE_ORDER, 
            RETRAIN_AMOUNT, [PREC_REDUCT for _ in ws_controller.model_layers], CLUST_MOD_FOCUS, CLUST_MOD_SPREAD, clust_alg=CLUST_ALG)
    
    return [individual.data['accuracy'], individual.data['compression']]

def fit_from_vals(data, targ_vals):

    return 1 / math.sqrt(pow(1 - (data['accuracy']/targ_vals[0]), 2) + pow(1 - (data['compression']/targ_vals[1]), 2))

Defining logging

In [ ]:
data = {
    'representation': [],
    'accuracy': [],
    'accuracy_loss': [],
    'compression': [],
    'share_t': [],
    'train_t': [],
    'acc_t': []
}

data_types = {
    'accuracy': 'float32',
    'accuracy_loss': 'float32',
    'compression': 'float32',
    'share_t': 'float32',
    'train_t': 'float32',
    'acc_t': 'float32'
}

rnd_data = pd.read_csv(SAVE_RND_FILE).astype(data_types) if os.path.exists(SAVE_RND_FILE) else pd.DataFrame(data).astype(data_types)

def logger_fc(rnd_controler:RandomController):
    global rnd_data

    new_data = copy.deepcopy(data)

    indiv = rnd_controler.current_indiv

    new_data['representation'].append(indiv.representation)
    new_data['accuracy'].append(indiv.data['accuracy'])
    new_data['accuracy_loss'].append(before_loss - indiv.data['accuracy'])
    new_data['compression'].append(indiv.data['compression'])
    new_data['share_t'].append(indiv.data['times']['share'])
    new_data['train_t'].append(indiv.data['times']['train'])
    new_data['acc_t'].append(indiv.data['times']['test'])

    # saving progress
    rnd_data = rnd_data.append(pd.DataFrame(new_data).astype(data_types))
    rnd_data.reset_index(drop=True, inplace=True)
    os.makedirs(os.path.dirname(SAVE_RND_FILE), exist_ok=True)
    rnd_data.to_csv(SAVE_RND_FILE, index=False)

Setting ranges with optimization

In [ ]:
lam_test_inp = lambda _ : get_accuracy(model, dataset.test_dl, DEVICE)

if RANGE_OPTIMIZATION:
    REPR_RANGES = ws_controller.get_optimized_layer_ranges(REPR_RANGES, lam_test_inp, RANGE_OPTIMIZATION_TRESHOLD, 
        savefile=RANGE_OPTIMIZATION_FILE, prec_rtype=PREC_REDUCT, clust_alg=CLUST_ALG)

for repr_range in REPR_RANGES:
    print(len(repr_range))

Running the search

In [ ]:
fit_controll = FitnessController(TARGET, fitness_vals_fc, fit_from_vals, target_max_offset=TARGET_UPDATE_OFFSET, 
    lock=LOCK_TARGET, target_limit=TARGET_LOW_LIMIT)
random = RandomController(REPR_RANGES, fit_controll)

if rnd_data.size != 0:
    random.load_from_pd(rnd_data, True)

random.run(NUM_INDIVIDUALS, logger_fc, verbose = True)

See output

In [ ]:
rnd_data

Plotting data

In [ ]:
plot_alcr(rnd_data)
plt.title('Random search on LeNet-5')

In [ ]:
fit_controll.targ